In [1]:
import torch
import wandb
import torchvision

import numpy as np

from data_generator import DataGenerator
from default_mnist_config import create_default_mnist_config
from diffusion import DiffusionRunner
from models.classifier import ResNet, ResidualBlock, ConditionalResNet

from matplotlib import pyplot as plt

import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [2]:
device = torch.device('cuda')
classifier_args = {
    "block": ResidualBlock,
    "layers": [2, 2, 2, 2]
}
noisy_classifier = ConditionalResNet(**classifier_args)
noisy_classifier.to(device)

noisy_classifier.load_state_dict(torch.load('./ddpm_checkpoints/classifier.pth'))

clean_classifier = ResNet(**classifier_args)
clean_classifier.to(device)

clean_classifier.load_state_dict(torch.load('./ddpm_checkpoints/clean_classifier.pth'))

<All keys matched successfully>

In [ ]:
noisy_classifier.eval()
clean_classifier.eval()

#### Создайте два семплемера картинок, безусловный и условный с методом .set_classifier

In [3]:
conditional_diffusion = DiffusionRunner(create_default_mnist_config(), eval=True)
conditional_diffusion.set_classifier(noisy_classifier, T=1.)

unconditional_diffusion = DiffusionRunner(create_default_mnist_config(), eval=True)

In [4]:
def get_pred_labels(images_normed: torch.Tensor):
    """
    predict labels for normed images 
    [-1, 1]
    """
    images_normed = images_normed.to(device)
    return clean_classifier(images_normed).argmax(dim=-1)


def calc_acc_clean_classifier(images_normed: torch.Tensor, labels: torch.Tensor):
    """
    calculate accuracy using clean classifier
    """
    labels = labels.to(device)
    preds = get_pred_labels(images_normed)
    acc = (preds==labels).sum().item() / labels.shape[0]
    return acc


def sample_images(diff_process, labels=None):
    images_tensor = diff_process.sample_images(batch_size=100, labels=labels)
    images_cpu = images_tensor.cpu()
    grid = torchvision.utils.make_grid(images_cpu, nrow=10).permute(1, 2, 0)
    grid = grid.data.numpy().astype(np.uint8)

    plt.imshow(grid)
    plt.show()
    return images_tensor


def cond_print_and_calc_acc(class_num: int):
    labels = class_num*torch.ones(100).long().to(device)
    images_tensor = sample_images(conditional_diffusion, labels)
    acc = calc_acc_clean_classifier((images_tensor - 127.5)/ 127.5, labels=labels)
    print('Accuracy: ', acc.item())
    return images_tensor

#### Посемплируйте наборы картинок всех классов обоими способами, изменить температуру можно с помощью повторного .set_classifier

In [ ]:
T = 1 # tochange
conditional_diffusion.set_classifier(noisy_classifier, T=T)

for i in range(10):
    cond_print_and_calc_acc(i)

In [ ]:
T = 1 # tochange
conditional_diffusion.set_classifier(noisy_classifier, T=T)

for i in range(10):
    cond_print_and_calc_acc(i)

> Как сильно влияет температура на отличимость семплов при условной генерации? Продеменстрируйте для разных T свои выводы.



> Какое качество получается у чистого классификатора при условной генерации? Попробуйте оценить на глаз качество чистого классификатора для безусловной генерации, объясните свои выводы.


### Теоретические выкладки

Прямой процесс задаётся стохастическим диффуром:
$$
    dx_t = -\frac{1}{2}\beta(t) x_t dt + \sqrt{\beta(t)} dW_t
$$
На семинаре по стохдиффурам решали аналогичное уравнение для $\beta(t) = \beta$, сделали замену $y = x exp(\beta t/2)$. 

---

Попробуем в нашем случае аналогично посмотреть на решение диффура без стохастической части:
\begin{gather*}
    dx_t = -\frac{1}{2}\beta(t)x_t dt\\
    \Downarrow\\
    \ln x_t = -\frac{1}{2}\underbrace{\int\limits_{0}^t \beta(s)ds}_{B(t)} + C
\end{gather*}

То есть $B(t)$ - первообразная $\beta(t)$, причем $B(0) = 0$.


Подставляя начальную точку получаем решение:
$$
    x_t = x_0 e^{-B(t)/2}
$$

---

Вернёмся к стохдиффуру, сделаем замену:
$$
    y_t = x_t e^{B(t)/2}
$$

По формуле Ито получаем стохдиффур на $y_t$:
\begin{gather*}
    dy_t = \left(e^{B(t)/2}\beta(t)/2 + -\beta(t)e^{B(t)/2}/2 + 0\right) + e^{B(t)/2}\sqrt{\beta(t)} dW_t\\
    dy_t = e^{B(t)/2}\sqrt{\beta(t)} dW_t
\end{gather*}

Благодаря тому, что мы подставили решение обычного диффура, нестохастическая часть сократилась. Аналогично семинару получаем:
\begin{gather*}
    \mathbb{E}[y_t] = y_0 = x_0\\
    \mathbb{D}[y_t] = \int\limits_{0}^t e^{-\Beta(t)} \beta(t)ds = \int\limits_{0}^{\Beta(t)} e^{-\Beta} d \Beta = 1 - e^{-\Beta(t)}
\end{gather*}

Отсюда получаем:

\begin{gather*}
    \mathbb{E}[x_t] = e^{-B(t)/2} x_0
    \mathbb{D}[x_t] = 1 - e^{-\Beta(t)}
\end{gather*}

В нашем случае $\beta(t) = \frac{\beta(1) - \beta(0)}{T}t + \beta(0)$ -- линейная функция. Тогда:
$$
    B(t) = \frac{\beta(1) - \beta(0)}{2T}t^2 +\beta(0)t
$$